### 3.3.1 파인콘 API 초기화

In [5]:
from pinecone import Pinecone

# Pinecone API 키 설정
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

### 3.3.2 벡터 데이터 추가

In [6]:
from pinecone import ServerlessSpec
index_lists = [item.get("name") for item in pc.list_indexes().get("indexes")]

index_name = "my-index"

if index_name not in index_lists:
    pc.create_index(
        name=index_name,
        dimension=3, # 벡터의 차원 수
        metric="cosine", # 유사도 측정 방식 (cosine, euclidean, dotproduct 중 선택)
        spec=ServerlessSpec(
            cloud="aws", # 사용할 클라우드: aws 또는 gcp
            region="us-east-1" # 리전 설정 (free tier에서 지원되는 리전을 사용해야 함)
        )
    )

In [7]:
index = pc.Index(index_name)

# 벡터 데이터 삽입
vectors = [
    [0.1, 0.2, 0.3], # 첫 번째 데이터의 벡터
    [0.4, 0.5, 0.6], # 두 번째 데이터의 벡터
    [0.7, 0.8, 0.9], # 세 번째 데이터의 벡터
]

# 벡터를 인덱스에 추가
ids = ["doc1", "doc2", "doc3"]
index.upsert([(id, vector) for id, vector in zip(ids, vectors)])
print("아이템 임베딩을 인덱스에 추가했습니다.")

아이템 임베딩을 인덱스에 추가했습니다.


### 3.3.3 벡터 검색

In [8]:
import json

# 유사 벡터 검색
results = index.query(
    vector=[[0.1, 0.2, 0.3]], # 검색할 쿼리 벡터
    top_k=2, # 가장 유사한 2개의 벡터 반환
    include_metadata=False
)

formatted_results = {
    "검색된 문서 ID": results["matches"][0]["id"],
    "유사도 거리": results["matches"][0]["score"]
}
print("\n유사한 벡터 검색 결과:")
print(json.dumps(formatted_results, indent=4, ensure_ascii=False))


유사한 벡터 검색 결과:
{
    "검색된 문서 ID": "doc1",
    "유사도 거리": 0.999999464
}


In [9]:
# 메타데이터와 함께 벡터 삽입
vectors = [
    ([0.1, 0.2, 0.3], {"category": "A", "year": 2020}),
    ([0.4, 0.5, 0.6], {"category": "B", "year": 2021}),
    ([0.7, 0.8, 0.9], {"category": "A", "year": 2022}),
]

ids = ["doc1", "doc2", "doc3"]
index.upsert([(id, vector, metadata) for id, (vector, metadata) in zip(ids, vectors)])

{'upserted_count': 3}

In [10]:
# 검색 벡터
query_vector = [0.1, 0.2, 0.25]

# 메타데이터 필터링 조건
filter_condition = {
    "category": {"$eq": "A"},
    "year": {"$gt": 2020}
}

# 검색
query_result = index.query(
    vector=query_vector,
    top_k=2,
    filter=filter_condition,
    include_metadata=True
)

In [11]:
formatted_results = {
    "검색된 문서 ID": query_result["matches"][0]["id"],
    "메타데이터": query_result["matches"][0]["metadata"]
}
print("\n유사한 벡터 검색 결과:")
print(json.dumps(formatted_results, indent=4, ensure_ascii=False))


유사한 벡터 검색 결과:
{
    "검색된 문서 ID": "doc3",
    "메타데이터": {
        "category": "A",
        "year": 2022.0
    }
}


### 3.3.5 임베딩 기반 라마인덱스 답변 생성

In [24]:
!pip install llama-index==0.11.11 \
    llama-index-vector-stores-pinecone==0.3.0 \
    llama-index-embeddings-huggingface==0.3.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
from pinecone import Pinecone
from llama_index.core.schema import Document
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import Document
import os

In [13]:
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)
index_name = "example"
spec = {
    "serverless": {
    "cloud": "aws",
    "region": "us-east-1"
    }
}
index_lists = [item.get("name") for item in pc.list_indexes().get("indexes")]
if index_name not in index_lists:
    pc.create_index(
        name=index_name,
        dimension=3,
        metric="cosine",
        spec=spec
    )
index = pc.Index(index_name)

# LlamaIndex에서 사용할 HuggingFace 임베딩 모델 설정
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

In [14]:
# 문서 데이터
documents = [
    "고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.",
    "강아지는 충성심이 강하고 친절한 동물로, 흔히 인간의 최고의 친구로 불립니다. 주로 애완동물로 기르고, 동반자로서 유명합니다.",
    "고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다."
]
ids = ["doc1", "doc2", "doc3"]

# 문서를 Document 형식으로 변환
nodes = [Document(text=doc, id_=doc_id) for doc, doc_id in zip(documents, ids)]

# Pinecone 벡터 스토어 생성
vector_store = PineconeVectorStore(pinecone_index=index)

# LlamaIndex의 VectorStoreIndex 생성
index = VectorStoreIndex.from_documents(nodes, vector_store=vector_store, embed_model= embed_model)

In [15]:
query_engine = index.as_query_engine()

# 질의 수행
query_text = "고양이에 대해 알려줘"
response = query_engine.query(query_text)

# 결과 출력
print("\n[질의 결과]")
print(response)


[질의 결과]
고양이는 독립적이고 깨끗한 동물로 알려져 있습니다. 주로 자신의 영역을 지키는 경계심이 강하며, 쾌활하고 호기심 많은 성격을 가지고 있습니다. 일반적으로 강아지와는 다르게 냥이라고도 불리며, 사람들 사이에서는 고양이는 독립적이면서도 사랑스러운 애완동물로 인식되고 있습니다.


### 3.3.6 라마인덱스 기반 답변 생성(임베딩 생략)

In [ ]:
vector_store = PineconeVectorStore(pinecone_index=index)

# LlamaIndex의 VectorStoreIndex 생성
index = VectorStoreIndex.from_documents(nodes, vector_store=vector_store)

# 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 질의 수행
query_text = "고양이에 대해 알려줘"
response = query_engine.query(query_text)

# 최종 응답 출력
print("[질의 결과]")
print(response)

# 응답 생성에 사용된 문서 확인
print("\n[응답에 사용된 문서]")
for i, node in enumerate(response.source_nodes, 1):
    print(f"{i}. {node.text}\n")